In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def if_same_array(px, py):
    num_mutched_elements = np.count_nonzero(px==py)
    if (num_mutched_elements== px.size and num_mutched_elements== py.size):
        return True
    else:
        return False
    
label_path = './label_7_37.png'
#label_path = './star.jpg'
img = cv2.imread(label_path)

pixel_dict = {'background':np.array([0, 0, 0]), 'stem':np.array([255, 255, 255])}

count_dict_length = 0
idx_cnt = 0
berry_idx = 'id_' + str(idx_cnt)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        for k in pixel_dict:
            if if_same_array(img[i][j], pixel_dict[k]):
                count_dict_length = 0
                flag = True
                break
            else:
                count_dict_length = count_dict_length + 1
            
        if flag:
            flag = False
            continue
            
        if(count_dict_length == len(pixel_dict)):
            #print(img[i][j])
            pixel_dict[berry_idx] = img[i][j]
            idx_cnt = idx_cnt + 1
            berry_idx = 'id_' + str(idx_cnt)
                

berry_mask_dict = {}
index_keys = [k for k in pixel_dict if 'id_' in k]
idx_counter=0

for k in index_keys:
    mask_coordinate = np.zeros((0, 2))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if(if_same_array(img[i][j], pixel_dict[k])):
                mask_coordinate = np.r_[mask_coordinate, np.array([[i, j]])]
    
    berry_mask_dict['id_' + str(idx_counter)] = mask_coordinate
    idx_counter = idx_counter + 1
                  
                
for k in berry_mask_dict:
    berry_mask_dict[k] = berry_mask_dict[k].astype(int)

from pascal_voc_writer import Writer

grey_img = cv2.imread(label_path, 0)
temp_img = np.zeros(grey_img.shape)

hough_threshold = 16

counter=0

writer = Writer(label_path, temp_img.shape[0], temp_img.shape[1])

for k in berry_mask_dict:
    
    empty_img = temp_img.copy()
    for l in range(len(berry_mask_dict[k])):
        empty_img[berry_mask_dict[k][l][0]][berry_mask_dict[k][l][1]] = 255
    empty_img = empty_img.astype(np.uint8)
         
    # 実ごとのバウンディングボックスを計算する箇所
    ret,thresh = cv2.threshold(empty_img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    
    # 実ごとにハフ変換で円を検出してtruncatedか否かを判断する箇所
    img = cv2.medianBlur(empty_img, 5)
    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                            param1=20,param2=hough_threshold,minRadius=0,maxRadius=0)
    
    if not (circles is None):
        truncated = False
    else:
        truncated = True
        
    # .xml ファイルに書き込むアノテーションの情報を追加
    writer.addObject('berry', x, y, x + w, y+h, truncated=truncated)

    counter = counter + 1
    
writer.save('./label_7_37.xml')